## Featurize and save Deml based featurization data
Data is featurized based on representations used in [Deml et al paper](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.93.085142). 

Note: Run this before `04.ipynb`!

In [1]:
import numpy as np
import pandas as pd
import os
import pymatgen as pmg

from matminer.utils.conversions import str_to_composition, composition_to_oxidcomposition
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers import composition as cf

from sklearn.preprocessing import PolynomialFeatures

from pymatgen import MPRester

### Loading deml_dataset.csv 

In [2]:
path = os.path.join(os.getcwd(), "oqmd_icsd_subset.pkl")
df = pd.read_pickle(path)

In [3]:
print ("Shape of data: ", df.shape)

Shape of data:  (36054, 11)


Drop unnecessary columns

In [4]:
df = df.drop(['band_gap', 'magnetic_moment', 'path', 'stability', 'structure', 'volume_pa', 'is_ICSD', 'structure_obj'], 1)

### Compute data representation

Compute pymatgen composition from compound formula

In [5]:
df['composition_obj'] = str_to_composition(df['composition'])

Compute ionic states

In [6]:
df['oxidation_states'] = composition_to_oxidcomposition(df['composition_obj'])

Remove compounds that cannot be featurized (due to unclear reasons)

In [7]:
df = df.reset_index(drop=True)

In [8]:
for i in [952, 1214, 1217, 1311, 1315, 1710, 1963]:
    df = df.drop([i, i])
df = df.reset_index(drop=True)

Adding a finite list of quantitative descriptors ([Deml et al 2016](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.93.085142))

In [11]:
%%time
ft = MultipleFeaturizer([cf.ElementProperty.from_preset('deml'), 
                         cf.TMetalFraction(),
                         cf.ValenceOrbital(),
                         cf.CationProperty.from_preset('deml'),
                         cf.OxidationStates.from_preset('deml'),
                         cf.ElectronAffinity(),
                         cf.ElectronegativityDiff()])
df = ft.featurize_dataframe(df, col_id='oxidation_states', ignore_errors=True)

CPU times: user 26.2 s, sys: 1.57 s, total: 27.8 s
Wall time: 1min 20s


Drop stats of f orbital valence electrons

In [12]:
df = df.drop(['frac f valence electrons', 'avg f valence electrons'], 1)

Calculate number of atoms in a formula unit

In [13]:
df['num_atoms'] = df['composition_obj'].apply(lambda x: x.num_atoms)

At this point, we should have 124 main descriptors (Deml's paper)

In [14]:
print ("Shape of data: ", df.drop(['composition', 'composition_obj', 'oxidation_states'], 1).shape)

Shape of data:  (36047, 125)


Fill in NaN values with zeros

In [15]:
df.fillna(value=0, inplace=True)

Sqrt and inverse of each term.

In [16]:
def inv(x):
    try:
        output = 1.0/x
    except:
        output = 0.0
    return output

In [17]:
col = df.drop(['composition', 'composition_obj', 'oxidation_states'], 1).columns
mean_col = []

In [18]:
for i in col:
    df["inverse %s"%i] = df[i].apply(lambda x: inv(x))
    df["sqrt %s"%i] = df[i].apply(lambda x: np.sqrt(x))
    if "mean" in i:
        mean_col.append(i)    

There should be an additional 248 terms added, coming up to a total of 372 descriptors.

In [19]:
print ("Shape of data: ", df.shape)

Shape of data:  (36047, 378)


Products of the primary (those without an asterisk) and stoichiometric weighted mean values. 

In [20]:
primary = ['num_atoms', 'transition metal fraction', 'avg anion electron affinity',
           'avg s valence electrons', 'avg p valence electrons', 
           'avg d valence electrons', 'frac s valence electrons', 
           'frac p valence electrons','frac d valence electrons']

In [21]:
product = df[mean_col + primary]
col = product.columns

Use PolynomialFeatures with degree 2 from scikit-learn package

In [22]:
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=True)
product = pd.DataFrame(poly.fit_transform(product))
product.columns = poly.get_feature_names(col)
product = product.drop(["1"]+ mean_col+primary, 1)
print (product.shape)

(36047, 496)


Merge into original dataframe

In [23]:
df[product.columns] = product

Final shape of data (Should have 4692 descriptors, but we're lacking a lot due to the products)

In [24]:
print ("Shape of featurized data: ", df.shape)
df.head(1)

Shape of featurized data:  (36047, 874)


,delta_e,total_energy,composition,composition_obj,oxidation_states,minimum atom_num,maximum atom_num,range atom_num,mean atom_num,std_dev atom_num,...,avg d valence electrons^2,avg d valence electrons frac s valence electrons,avg d valence electrons frac p valence electrons,avg d valence electrons frac d valence electrons,frac s valence electrons^2,frac s valence electrons frac p valence electrons,frac s valence electrons frac d valence electrons,frac p valence electrons^2,frac p valence electrons frac d valence electrons,frac d valence electrons^2
0,-3.579764,-7.936143,Ac2O3,"(Ac, O)","(Ac3+, O2-)",0.0,0.0,0.0,0.0,0.0,...,0.16,0.166667,0.2,0.033333,0.173611,0.208333,0.034722,0.25,0.041667,0.006944


Saving featurized data to pickle file

In [25]:
df.to_pickle('./deml_featurized_data.pkl')